# TVM, load transformer

In [1]:
!pwd

/Users/isong/Downloads/ml/sc/xilinx/Github/transformer_simple/src/python


## Test run

In [2]:
!python ./experiments/classify.py -e 1 -t -d1 -H1 -D -m single_transformer.pt

/Users/isong/anaconda3x/envs/nlu/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
OPTIONS  Namespace(batch_size=4, debug=True, depth=1, embedding_size=128, final=False, gradient_clipping=1.0, lr=0.0001, lr_warmup=10000, max_length=512, max_pool=False, model_name='single_transformer.pt', num_epochs=1, num_heads=1, seed=1, tb_dir='./runs', tiny=True, vocab_size=50000)
/Users/isong/anaconda3x/envs/nlu/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will

100%|███████████████████████████████████████████| 63/63 [00:04<00:00, 13.76it/s]
-- validation accuracy 0.46
Save model to saved_model/single_transformer.pt
Load model to saved_model/single_transformer.pt
Model's state_dict:
token_embedding.weight 	 torch.Size([50000, 128])
pos_embedding.weight 	 torch.Size([512, 128])
trfm_blocks.0.mha.attentions.0.toqueries.weight 	 torch.Size([128, 128])
trfm_blocks.0.mha.attentions.0.toqueries.bias 	 torch.Size([128])
trfm_blocks.0.mha.attentions.0.tokeys.weight 	 torch.Size([128, 128])
trfm_blocks.0.mha.attentions.0.tokeys.bias 	 torch.Size([128])
trfm_blocks.0.mha.attentions.0.tovalues.weight 	 torch.Size([128, 128])
trfm_blocks.0.mha.attentions.0.tovalues.bias 	 torch.Size([128])
trfm_blocks.0.mha.w_o.0.weight 	 torch.Size([128, 128])
trfm_blocks.0.mha.w_o.0.bias 	 torch.Size([128])
trfm_blocks.0.norm1.weight 	 torch.Size([128])
trfm_blocks.0.norm1.bias 	 torch.Size([128])
trfm_blocks.0.norm2.weight 	 torch.Size([128])
trfm_blocks.0.norm2.bias 	

state 	 {0: {'step': 63, 'exp_avg': tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.1634e-03,  9.5109e-04, -4.7927e-04,  ...,  7.0324e-04,
         -4.7953e-04,  1.2503e-03],
        [-9.8753e-05,  8.8170e-05, -3.6461e-05,  ...,  8.9684e-05,
         -5.3466e-05,  1.2733e-04],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]]), 'exp_avg_sq': tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [6.2477e-07, 4.3881e-07, 1.1684e-07,  ..., 2.2475e-07, 1.1106e-07,
         7.2727e-07],
        [8.9104e-09, 6.7384e-09, 1.3663e-09,  ..., 7.6352e-09, 2.9266e-09,
         1.6498e-08],
        ...,
        [0.00

## Load pytorch model to tvm 
- [tvm reference](https://tvm.apache.org/docs/tutorials/frontend/from_pytorch.html#sphx-glr-tutorials-frontend-from-pytorch-py)

In [71]:
# tvm modules

import tvm
from tvm import relay

import numpy as np

from tvm.contrib.download import download_testdata

# PyTorch imports
import torch
import torchvision

In [72]:
# transformer modules

import transformer_simple
import classifier
import util

In [73]:
model_name = "transformer"
# single transformer
mx = 512
embedding_size = 128
vocab_size = 50000
NUM_CLS = 2
max_pool = False
num_heads = 1
depth = 1

PATH = 'saved_model/single_transformer1.pt'

model = classifier.TransformerSimpleClassify(n_seq=mx, dim_emb=embedding_size, dim_internal=embedding_size, \
                                                         num_tokens=vocab_size, num_classes=NUM_CLS, max_pool=max_pool, \
                                                         heads=num_heads, depth=depth)
model.load_state_dict(torch.load(PATH))
model = model.eval()

In [74]:
# We grab the TorchScripted model via tracing
input_shape = [4, 498]
input_data = torch.randint(0, vocab_size, input_shape)
scripted_model = torch.jit.trace(model, input_data).eval()

## Import the graph to Relay

In [105]:
input_name = "input0"
shape_list = [(input_name, input_shape)]
mod, params = relay.frontend.from_pytorch(scripted_model, shape_list)

## Relay Build

In [106]:
target = "llvm"
target_host = "llvm"
ctx = tvm.cpu(0)
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, target_host=target_host, params=params)

## Execute the portable graph on TVM

In [107]:
from tvm.contrib import graph_runtime

m = graph_runtime.GraphModule(lib["default"](ctx))
# Set inputs
m.set_input(input_name, tvm.nd.array(input_data))
# Execute
m.run()
# Get outputs
tvm_output = m.get_output(0)

In [108]:
type(tvm_output)

tvm.runtime.ndarray.NDArray

In [109]:
tvm_output

<tvm.nd.NDArray shape=(4, 2), cpu(0)>
array([[-0.7326672 , -0.6551298 ],
       [-0.7252913 , -0.66200423],
       [-0.73297906, -0.6548413 ],
       [-0.73238647, -0.65538967]], dtype=float32)

In [110]:
input_data

tensor([[ 3765, 35163, 22374,  ..., 10840, 34026, 31777],
        [16322, 12776, 30706,  ...,  7692, 48182, 26912],
        [ 9161, 34760, 14639,  ..., 29835, 30657, 18810],
        [17196, 32012, 24887,  ..., 49110, 43611,  2869]])

In [111]:
type(input_data)

torch.Tensor

In [112]:
INPUT_DATA_PATH='saved_model/input_data.pt'
torch.save(input_data, INPUT_DATA_PATH)

In [113]:
np_output = tvm_output.asnumpy()

In [114]:
torch_output = torch.from_numpy(np_output)

In [115]:
torch_output

tensor([[-0.7327, -0.6551],
        [-0.7253, -0.6620],
        [-0.7330, -0.6548],
        [-0.7324, -0.6554]])

In [116]:
OUTPUT_DATA_PATH='saved_model/output_data.pt'
torch.save(torch_output, OUTPUT_DATA_PATH)

## Generate code (WIP)

In [117]:
#mhost = tvm.build(mod, target=target)
print(mod.astext(show_meta_data=True))

#[version = "0.0.5"]
type tensor_float16_t {
  tensor_nil_float16,
  tensor0_float16(float16),
  tensor1_float16(Tensor[(?), float16]),
  tensor2_float16(Tensor[(?, ?), float16]),
  tensor3_float16(Tensor[(?, ?, ?), float16]),
  tensor4_float16(Tensor[(?, ?, ?, ?), float16]),
  tensor5_float16(Tensor[(?, ?, ?, ?, ?), float16]),
  tensor6_float16(Tensor[(?, ?, ?, ?, ?, ?), float16]),
}

type Tree[A] {
  Rose(A, List[Tree[A]]),
}

type tensor_float64_t {
  tensor_nil_float64,
  tensor0_float64(float64),
  tensor1_float64(Tensor[(?), float64]),
  tensor2_float64(Tensor[(?, ?), float64]),
  tensor3_float64(Tensor[(?, ?, ?), float64]),
  tensor4_float64(Tensor[(?, ?, ?, ?), float64]),
  tensor5_float64(Tensor[(?, ?, ?, ?, ?), float64]),
  tensor6_float64(Tensor[(?, ?, ?, ?, ?, ?), float64]),
}

type List[A] {
  Cons(A, List[A]),
  Nil,
}

type tensor_uint16_t {
  tensor_nil_uint16,
  tensor0_uint16(uint16),
  tensor1_uint16(Tensor[(?), uint16]),
  tensor2_uint16(Tensor[(?, ?), uint16]),
  t

## Save and Load Compiled Module

In [118]:
# save the graph, lib and params into separate files
from tvm.contrib import util

LIB_PATH='saved_model/deploy_lib.tar'
lib.export_library(LIB_PATH)


In [119]:
LIB_PATH 

'saved_model/deploy_lib.tar'

In [120]:
# load the module back.
loaded_lib = tvm.runtime.load_module(LIB_PATH)


m = graph_runtime.GraphModule(loaded_lib["default"](ctx))
# Set inputs
m.set_input(input_name, tvm.nd.array(input_data))
# Execute
m.run()
# Get outputs
tvm_output = m.get_output(0)


In [121]:
tvm_output

<tvm.nd.NDArray shape=(4, 2), cpu(0)>
array([[-0.7326672 , -0.6551298 ],
       [-0.7252913 , -0.66200423],
       [-0.73297906, -0.6548413 ],
       [-0.73238647, -0.65538967]], dtype=float32)